In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.decomposition import PCA

In [2]:
df_train = pd.read_csv('secom_train_data.tsv', delimiter='\t')
df_test = pd.read_csv('secom_production_data_.csv')

In [3]:
df_train = pd.DataFrame(df_train)
df_train.head()

,failure,timestamp,s1,s2,s3,s4,s5,s6,s7,s8,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,-1,2008/07/19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,-1,2008/07/19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,1,2008/07/19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,-1,2008/07/19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,-1,2008/07/19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [4]:
df_test = pd.DataFrame(df_test)
df_test.head()

,failure,timestamp,s1,s2,s3,s4,s5,s6,s7,s8,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,NaN,2008/10/7 13:10,3052.98,2515.51,2172.8111,969.3436,1.2736,100.0,102.7367,0.1243,...,0.0059,52.7014,0.5081,0.0158,0.0037,3.1106,0.0302,0.0159,0.0059,52.7014
1,NaN,2008/10/7 13:48,2870.76,2583.56,2192.1889,1435.9611,2.3870,100.0,107.3989,0.1229,...,0.0063,193.8286,0.5010,0.0127,0.0037,2.5344,0.0111,0.0215,0.0063,193.8286
2,NaN,2008/10/7 14:35,3014.77,2493.11,2210.7333,1368.7060,2.2449,100.0,99.4200,0.1228,...,0.0100,60.5430,0.4987,0.0183,0.0041,3.6769,0.0458,0.0277,0.0100,60.5430
3,NaN,2008/10/7 16:32,3027.72,2430.70,2192.1889,1435.9611,2.3870,100.0,107.3989,0.1229,...,NaN,NaN,0.5046,0.0125,0.0032,2.4810,0.0458,0.0277,0.0100,60.5430
4,NaN,2008/10/7 17:06,2893.19,2498.09,2200.2111,1048.3108,1.6485,100.0,98.3222,0.1214,...,NaN,NaN,0.4954,0.0106,0.0030,2.1397,0.0458,0.0277,0.0100,60.5430


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Columns: 592 entries, failure to s590
dtypes: float64(586), int64(5), object(1)
memory usage: 5.4+ MB


欠損値を列の平均値で埋める(or median or mode)

In [6]:
df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

In [7]:
df_train = df_train.drop(columns=['timestamp'])
df_test = df_test.drop(columns=['timestamp']) 
df_train_feature = df_train.drop(columns=['failure'])
df_test_feature = df_test.drop(columns=['failure'])
df_train_feature.head()

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,0.005283,92.107101,0.5005,0.0118,0.0035,2.3630,0.02201,0.016233,0.0052,96.330285
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.006000,208.204500,0.5019,0.0223,0.0055,4.4447,0.00960,0.020100,0.0060,208.204500
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.014800,82.860200,0.4958,0.0157,0.0039,3.1745,0.05840,0.048400,0.0148,82.860200
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.004400,73.843200,0.4990,0.0103,0.0025,2.0544,0.02020,0.014900,0.0044,73.843200
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,0.005283,92.107101,0.4800,0.4766,0.1045,99.3032,0.02020,0.014900,0.0044,73.843200


In [8]:
pca = PCA()
pca.fit(df_train_feature)
pca.fit(df_test_feature)
train_feature_ = pca.transform(df_train_feature)
test_feature_ = pca.transform(df_test_feature)
train_feature_ = pd.DataFrame(train_feature_)
test_feature_ = pd.DataFrame(test_feature_)
train_feature_.head()

,0,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
0,-6401.795297,-2308.570063,907.103098,-302.130780,-117.442366,51.926666,-104.494209,321.066067,53.472227,930.755562,...,0.017631,-0.014025,0.042855,-0.058651,0.016736,0.018270,0.059260,-0.011973,-0.022631,-0.020479
1,-3243.550410,-3030.935694,-1569.066497,260.843005,-185.405395,-75.339876,14.197430,-402.937828,10.096726,887.163912,...,0.010760,0.001393,0.002514,-0.020265,0.013789,0.028548,0.022928,0.016231,0.005621,-0.018765
2,-4503.925685,-2370.542620,-1755.935841,-328.370272,-171.556191,166.784021,-67.859138,-125.288316,-173.141689,-439.042901,...,0.961864,-0.839662,-1.276594,0.723680,-0.392226,-0.669028,0.467609,0.892594,0.003707,1.022653
3,-3221.550153,378.940071,-982.154702,-665.626186,-56.580124,-537.340323,-90.325225,144.439161,-156.915813,319.984987,...,1.121202,-0.833338,-1.376558,0.769442,-0.364877,-0.573185,0.464826,0.768918,-0.012080,1.061649
4,-5159.069394,-1583.940386,427.344038,-257.146430,-130.885336,-179.213490,-33.228276,-56.904948,-95.356270,185.002454,...,0.109777,0.031836,-0.031716,-0.003848,-0.000408,-0.003419,0.067476,-0.150986,-0.018928,0.006835


In [9]:
# df_train_feature = pd.DataFrame(train_feature_, columns=["PC{}".format(x + 1) for x in range(len(train_feature_.columns))])
# df_test_feature = pd.DataFrame(test_feature_, columns=["PC{}".format(x + 1) for x in range(len(test_feature_.columns))])
# df_train_feature.head()

特徴抽出

SVM　学習

In [10]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_feature_, df_train['failure'])

C:\Users\tamaru\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [11]:
df_test.head()

,failure,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,NaN,3052.98,2515.51,2172.8111,969.3436,1.2736,100.0,102.7367,0.1243,1.4065,...,0.005900,52.701400,0.5081,0.0158,0.0037,3.1106,0.0302,0.0159,0.0059,52.7014
1,NaN,2870.76,2583.56,2192.1889,1435.9611,2.3870,100.0,107.3989,0.1229,1.4719,...,0.006300,193.828600,0.5010,0.0127,0.0037,2.5344,0.0111,0.0215,0.0063,193.8286
2,NaN,3014.77,2493.11,2210.7333,1368.7060,2.2449,100.0,99.4200,0.1228,1.6010,...,0.010000,60.543000,0.4987,0.0183,0.0041,3.6769,0.0458,0.0277,0.0100,60.5430
3,NaN,3027.72,2430.70,2192.1889,1435.9611,2.3870,100.0,107.3989,0.1229,1.5697,...,0.005672,112.114068,0.5046,0.0125,0.0032,2.4810,0.0458,0.0277,0.0100,60.5430
4,NaN,2893.19,2498.09,2200.2111,1048.3108,1.6485,100.0,98.3222,0.1214,1.5471,...,0.005672,112.114068,0.4954,0.0106,0.0030,2.1397,0.0458,0.0277,0.0100,60.5430


In [16]:
predict = clf.predict(test_feature_)
predict = pd.DataFrame(predict)
predict.head()

,0
0,-1
1,-1
2,-1
3,-1
4,-1


In [ ]:
predict.to_csv("predict.csv", header=False, index=False)